In [53]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

os.chdir("/Users/davidcoope/Desktop/BayesianOptimiser")
import BOPackage

In [54]:
def WeldedBeam(X):
    x1 = X[:, 0]
    x2 = X[:, 1]
    x3 = X[:, 2]
    x4 = X[:, 3]
    
    # Calculate the objective function value
    y = 1.10471 * x1**2 * x2 + 0.04811 * x3 * x4 * (x2 + 14)
    
    # Calculate constraint components
    tau1 = 6000 / (np.sqrt(2) * x1 * x2)
    tau2 = 504000 / (x3**2 * x4)
    tau = np.sqrt(tau1**2 + tau2**2 + (x2 * tau1 * tau2) / np.sqrt(0.25 * (x2**2) + (x1 + x3)**2))
    
    sigma = 504000 / (x3**2 * x4)
    
    Pc = 64746.022 * (1 - 0.0282346 * x3) * x3 * x4**3
    
    delta = 2.1952 / (x3 * x4)
    
    # Apply the constraints
    constraint1 = tau - 13000 > 0
    constraint2 = sigma - 30000 > 0
    constraint3 = x1 - x4 > 0
    constraint4 = 6000 - Pc > 0
    constraint5 = delta - 0.25 > 0
    
    # Apply penalties for constraint violations
    y[constraint1] = np.inf
    y[constraint2] = np.inf
    y[constraint3] = np.inf
    y[constraint4] = np.inf
    y[constraint5] = np.inf
    
    # Make function negative to find the minimum
    y = -y.reshape(-1, 1)
    
    return y

bounds = np.array([[0.125, 10], [0.125, 10], [0.1, 10], [0.1, 10]])
length_scale = 2.5
acquisition_samples = 1000
ObjectiveFunction = WeldedBeam

In [55]:
# X1 = np.linspace(bounds[0,0], bounds[0,1], 501, endpoint=True)
# X2 = np.linspace(bounds[1,0], bounds[1,1], 501, endpoint=True)

# # Create the meshgrid
# X1_grid, X2_grid = np.meshgrid(X1, X2)

# # Combine the grid into an n**2 x 2 matrix
# candidate_x = np.c_[X1_grid.ravel(), X2_grid.ravel()]

# result = ObjectiveFunction(candidate_x).reshape(X1_grid.shape)

# # Plot the contour
# contour = plt.contourf(X1, X2, result, levels=50)

# # Add a color bar
# plt.colorbar(contour)

# # Show the plot
# plt.show()

In [56]:
"""
Configure the optimiser.
"""
random_seed = 50

Kernel = BOPackage.RBF_Kernel

AcquisitionFunction = BOPackage.UpperConfidenceBound
max_kappa = 1
min_kappa = 0.05

reduce_bounds_factor = 0.5

# log_path = '/Users/davidcoope/Desktop/BayesianOptimiser/BraninHoo/BOlog.log'

# if os.path.exists(log_path):   
#     subprocess.run(['rm', '-rf', log_path], check=True, text=True, capture_output=True)  

In [57]:
"""
Configure the optimiser for the standard configuration.
"""

dynamic_bounds = False

# Create the object
bo1 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  acquisition_samples=acquisition_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=None,
                  first_reduce_bounds=None,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo1.GetRandomXBatch(1)
raw_y = ObjectiveFunction(raw_X)
bo1.UpdateData(raw_X, raw_y)

for i in range(299):
    raw_X = bo1.GetNextX(kappa=0.1)
    raw_y = ObjectiveFunction(np.array(raw_X))
    bo1.UpdateData(raw_X, raw_y)

bo1.PrintCurrentStatus()
print(bo1.bounds)
# BOPackage.PlotData(bo1)

BO1 = bo1.BestData()[1][0]

Current best y value was [-11.75033505]; the corresponding X values were [0.25101839 5.46411892 6.16908927 1.96819835]
[[ 0.125 10.   ]
 [ 0.125 10.   ]
 [ 0.1   10.   ]
 [ 0.1   10.   ]]


In [58]:
"""
Configure the optimiser with bounds.
"""

dynamic_bounds = True
iterations_between_reducing_bounds = 30
first_reduce_bounds = 100  

# Create the object
bo2 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  acquisition_samples=acquisition_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=iterations_between_reducing_bounds,
                  first_reduce_bounds=first_reduce_bounds,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo2.GetRandomXBatch(1)
raw_y = ObjectiveFunction(raw_X)
bo2.UpdateData(raw_X, raw_y)

for i in range(299):
    raw_X = bo2.GetNextX(kappa=0.1)
    raw_y = ObjectiveFunction(np.array(raw_X))
    bo2.UpdateData(raw_X, raw_y)

bo2.PrintCurrentStatus()
print(bo2.bounds)
# BOPackage.PlotData(bo2)

BO2 = bo2.BestData()[1][0]

Current best y value was [-9.4684092]; the corresponding X values were [0.18049582 7.54286653 6.13781647 1.44574152]
The bounds have been reduced 5 times
[[0.12797424 0.43656799]
 [7.30551841 7.61411216]
 [6.03968038 6.34905538]
 [1.2906355  1.6000105 ]]


In [59]:
"""
Configure the optimiser with batch sampling.
"""

dynamic_bounds = False

# Create the object
bo3 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  acquisition_samples=acquisition_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=iterations_between_reducing_bounds,
                  first_reduce_bounds=first_reduce_bounds,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo3.GetRandomXBatch(10)
raw_y = ObjectiveFunction(raw_X)
bo3.UpdateData(raw_X, raw_y)

for i in range(29):
    raw_X = bo3.GetNextXBatch(10, sub_batch_size=10, max_kappa=max_kappa, min_kappa=min_kappa)
    raw_y = ObjectiveFunction(raw_X)
    bo3.UpdateData(raw_X, raw_y)

bo3.PrintCurrentStatus()
print(bo3.bounds)
# BOPackage.PlotData(bo3)

BO3 = bo3.BestData()[1][0]

Current best y value was [-12.45941768]; the corresponding X values were [1.12416584 1.20699505 7.54175373 1.95272196]
[[ 0.125 10.   ]
 [ 0.125 10.   ]
 [ 0.1   10.   ]
 [ 0.1   10.   ]]


In [60]:
"""
Configure the optimiser with sub-batch sampling and bounds reduction.
"""

dynamic_bounds = True
iterations_between_reducing_bounds = 3
first_reduce_bounds = 50 

# Create the object
bo6 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  acquisition_samples=acquisition_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=iterations_between_reducing_bounds,
                  first_reduce_bounds=first_reduce_bounds,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo6.GetRandomXBatch(10)
raw_y = ObjectiveFunction(raw_X)
bo6.UpdateData(raw_X, raw_y)

for i in range(29):
    raw_X = bo6.GetNextXBatch(10, max_kappa=max_kappa, min_kappa=min_kappa)
    raw_y = ObjectiveFunction(raw_X)
    bo6.UpdateData(raw_X, raw_y)

bo6.PrintCurrentStatus()
print(bo6.bounds)
# BOPackage.PlotData(bo5)

BO6 = bo6.BestData()[1][0]

Current best y value was [-12.45941768]; the corresponding X values were [1.12416584 1.20699505 7.54175373 1.95272196]
The bounds have been reduced 0 times
[[ 0.125 10.   ]
 [ 0.125 10.   ]
 [ 0.1   10.   ]
 [ 0.1   10.   ]]


In [61]:
"""
Configure the optimiser with sub-batch sampling.
"""

dynamic_bounds = False

# Create the object
bo4 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  acquisition_samples=acquisition_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=iterations_between_reducing_bounds,
                  first_reduce_bounds=first_reduce_bounds,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo4.GetRandomXBatch(10)
raw_y = ObjectiveFunction(raw_X)
bo4.UpdateData(raw_X, raw_y)

for i in range(29):
    raw_X = bo4.GetNextXBatch(10, sub_batch_size=5, max_kappa=max_kappa, min_kappa=min_kappa)
    raw_y = ObjectiveFunction(raw_X)
    bo4.UpdateData(raw_X, raw_y)

bo4.PrintCurrentStatus()
print(bo4.bounds)
# BOPackage.PlotData(bo4)

BO4 = bo4.BestData()[1][0]

Current best y value was [-12.45941768]; the corresponding X values were [1.12416584 1.20699505 7.54175373 1.95272196]
[[ 0.125 10.   ]
 [ 0.125 10.   ]
 [ 0.1   10.   ]
 [ 0.1   10.   ]]


In [62]:
"""
Configure the optimiser with sub-batch sampling and bounds reduction.
"""

dynamic_bounds = True
iterations_between_reducing_bounds = 3
first_reduce_bounds = 10 

# Create the object
bo5 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  acquisition_samples=acquisition_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=iterations_between_reducing_bounds,
                  first_reduce_bounds=first_reduce_bounds,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo5.GetRandomXBatch(10)
raw_y = ObjectiveFunction(raw_X)
bo5.UpdateData(raw_X, raw_y)

for i in range(29):
    raw_X = bo5.GetNextXBatch(10, sub_batch_size=5, max_kappa=max_kappa, min_kappa=min_kappa)
    raw_y = ObjectiveFunction(raw_X)
    bo5.UpdateData(raw_X, raw_y)
    

bo5.PrintCurrentStatus()
print(bo5.bounds)
# BOPackage.PlotData(bo5)

BO5 = bo5.BestData()[1][0]

Current best y value was [-6.75883501]; the corresponding X values were [0.63211338 0.74636543 8.67623654 1.04452223]
The bounds have been reduced 4 times
[[0.32351963 0.94070713]
 [0.43777168 1.05495918]
 [8.65112071 9.26987071]
 [0.96996464 1.58871464]]


In [63]:
"""
Configure the optimiser with sub-batch sampling.
"""

dynamic_bounds = False

# Create the object
bo7 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  acquisition_samples=acquisition_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=iterations_between_reducing_bounds,
                  first_reduce_bounds=first_reduce_bounds,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo7.GetRandomXBatch(10)
raw_y = ObjectiveFunction(raw_X)
bo7.UpdateData(raw_X, raw_y)

for i in range(29):
    raw_X = bo4.GetNextXBatch(10, sub_batch_size=2, max_kappa=max_kappa, min_kappa=min_kappa)
    raw_y = ObjectiveFunction(raw_X)
    bo7.UpdateData(raw_X, raw_y)

bo7.PrintCurrentStatus()
print(bo7.bounds)
# BOPackage.PlotData(bo4)

BO7 = bo7.BestData()[1][0]

Current best y value was [-12.45941768]; the corresponding X values were [1.12416584 1.20699505 7.54175373 1.95272196]
[[ 0.125 10.   ]
 [ 0.125 10.   ]
 [ 0.1   10.   ]
 [ 0.1   10.   ]]


In [64]:
"""
Configure the optimiser with sub-batch sampling and bounds reduction.
"""

dynamic_bounds = True
iterations_between_reducing_bounds = 3
first_reduce_bounds = 10 

# Create the object
bo8 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  acquisition_samples=acquisition_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=iterations_between_reducing_bounds,
                  first_reduce_bounds=first_reduce_bounds,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo8.GetRandomXBatch(10)
raw_y = ObjectiveFunction(raw_X)
bo8.UpdateData(raw_X, raw_y)

for i in range(29):
    raw_X = bo8.GetNextXBatch(10, sub_batch_size=2, max_kappa=max_kappa, min_kappa=min_kappa)
    raw_y = ObjectiveFunction(raw_X)
    bo8.UpdateData(raw_X, raw_y)
    

bo8.PrintCurrentStatus()
print(bo8.bounds)
# BOPackage.PlotData(bo5)

BO8 = bo8.BestData()[1][0]

Current best y value was [-6.75883501]; the corresponding X values were [0.63211338 0.74636543 8.67623654 1.04452223]
The bounds have been reduced 4 times
[[0.32351963 0.94070713]
 [0.43777168 1.05495918]
 [8.65112071 9.26987071]
 [0.96996464 1.58871464]]


In [65]:
print(BO1, BO2, BO3, BO6, BO4, BO5, BO7, BO8)

[-11.75033505] [-9.4684092] [-12.45941768] [-12.45941768] [-12.45941768] [-6.75883501] [-12.45941768] [-6.75883501]
